In [10]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,RMSprop
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import to_categorical


In [11]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

Num GPUs Available:  1
/device:GPU:0


In [24]:
# Load data into pandas DataFrame
df = pd.read_parquet('unscaled_train_data.parquet')

# Define the feature and target columns
feature_cols = df.columns[:-1]
target_col = df.columns[-1]
O = len(df[target_col].unique())
I = len(feature_cols)

# Define a function to create the Keras model
def create_model(hidden_size:int,lr:float=None,m:float=None):
    model = Sequential()
    model.add(Dense(hidden_size, input_dim=len(feature_cols), activation='sigmoid'))
    model.add(Dense(O, activation='softmax'))
    if lr and m:
        optimizer = SGD(learning_rate=lr,momentum=m)
    else:
        optimizer = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_crossentropy','mean_squared_error','accuracy'])
    return model

# Create a 5-fold cross-validation object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Encode data
y = to_categorical(df[target_col])

In [13]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [14]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [15]:
def wrap_cross_validate(pipe:Pipeline):
    # Use cross_val_score to evaluate the pipeline on the data
    scores = cross_validate(pipe, df[feature_cols], y,
    cv=kf,
    #n_jobs=2,
    scoring=
    {
        'accuracy': 'accuracy',
        'loss': 'neg_log_loss',
        'mse': 'neg_mean_squared_error'
    })
    return scores

def print_scores(scores):
    ce_scores = scores['test_loss']
    mse_scores = scores['test_mse']
    acc_scores = scores['test_accuracy']

    # Print the mean and standard deviation of the scores
    print('Cross Entropy: {:.3f} (+/- {:.3f})'.format(-ce_scores.mean(), ce_scores.std()))
    print('MSE: {:.3f} (+/- {:.3f})'.format(-mse_scores.mean(), mse_scores.std()))
    print('Accuracy: {:.3f} (+/- {:.3f})'.format(acc_scores.mean(), acc_scores.std()))

In [16]:
# Create a pipeline with a StandardScaler and a Keras model
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KerasClassifier(model=create_model(I), epochs=10, batch_size=64))
])

scores_1 = wrap_cross_validate(pipe)

INFO:tensorflow:Assets written to: C:\Users\tomi\AppData\Local\Temp\tmp7_rddjze\assets
Epoch 1/10
2071/2071 [==============================] - 11s 5ms/step - loss: 0.2845 - categorical_crossentropy: 0.2845 - mean_squared_error: 0.0281 - accuracy: 0.9078
Epoch 2/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.1188 - categorical_crossentropy: 0.1188 - mean_squared_error: 0.0113 - accuracy: 0.9637
Epoch 3/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.0983 - categorical_crossentropy: 0.0983 - mean_squared_error: 0.0093 - accuracy: 0.9704
Epoch 4/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.0915 - categorical_crossentropy: 0.0915 - mean_squared_error: 0.0086 - accuracy: 0.9725
Epoch 5/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.0876 - categorical_crossentropy: 0.0876 - mean_squared_error: 0.0082 - accuracy: 0.9739
Epoch 6/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0

In [17]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KerasClassifier(model=create_model(int((I+O)/2)), epochs=10, batch_size=64))
])

scores_2 = wrap_cross_validate(pipe)

INFO:tensorflow:Assets written to: C:\Users\tomi\AppData\Local\Temp\tmpk6kpgp8h\assets
Epoch 1/10
2071/2071 [==============================] - 11s 5ms/step - loss: 0.3296 - categorical_crossentropy: 0.3296 - mean_squared_error: 0.0324 - accuracy: 0.8953
Epoch 2/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.1573 - categorical_crossentropy: 0.1573 - mean_squared_error: 0.0151 - accuracy: 0.9509
Epoch 3/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.1370 - categorical_crossentropy: 0.1370 - mean_squared_error: 0.0130 - accuracy: 0.9583
Epoch 4/10
2071/2071 [==============================] - 11s 5ms/step - loss: 0.1294 - categorical_crossentropy: 0.1293 - mean_squared_error: 0.0121 - accuracy: 0.9612
Epoch 5/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.1256 - categorical_crossentropy: 0.1255 - mean_squared_error: 0.0117 - accuracy: 0.9629
Epoch 6/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0

In [18]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KerasClassifier(model=create_model(I+O), epochs=10, batch_size=64))
])

scores_3 = wrap_cross_validate(pipe)

INFO:tensorflow:Assets written to: C:\Users\tomi\AppData\Local\Temp\tmpn7wtj9xt\assets
Epoch 1/10
2071/2071 [==============================] - 11s 5ms/step - loss: 0.2610 - categorical_crossentropy: 0.2610 - mean_squared_error: 0.0260 - accuracy: 0.9143
Epoch 2/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.1037 - categorical_crossentropy: 0.1037 - mean_squared_error: 0.0101 - accuracy: 0.9673
Epoch 3/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.0853 - categorical_crossentropy: 0.0853 - mean_squared_error: 0.0082 - accuracy: 0.9737
Epoch 4/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.0767 - categorical_crossentropy: 0.0767 - mean_squared_error: 0.0073 - accuracy: 0.9765
Epoch 5/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0.0715 - categorical_crossentropy: 0.0715 - mean_squared_error: 0.0068 - accuracy: 0.9781
Epoch 6/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0

In [19]:
print_scores(scores_1)
print_scores(scores_2)
print_scores(scores_3)

Cross Entropy: 0.079 (+/- 0.001)
MSE: 0.009 (+/- 0.000)
Accuracy: 0.976 (+/- 0.000)
Cross Entropy: 0.116 (+/- 0.002)
MSE: 0.014 (+/- 0.000)
Accuracy: 0.966 (+/- 0.001)
Cross Entropy: 0.063 (+/- 0.005)
MSE: 0.008 (+/- 0.001)
Accuracy: 0.981 (+/- 0.002)


In [25]:
test_matrice=[
    {'lr':0.001,'m':0.2},
    {'lr':0.001,'m':0.6},
    {'lr':0.05,'m':0.6},
    {'lr':0.1,'m':0.6},
]
results = []
for cond in test_matrice:
    pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KerasClassifier(model=create_model(I+O,lr=cond['lr'],m=cond['m']), epochs=10, batch_size=64))
    ])

    score = wrap_cross_validate(pipe)
    results.append(
        {
            'condition':cond,
            'score':score,
            'keras':pipe["clf"]
        }
    )

INFO:tensorflow:Assets written to: C:\Users\tomi\AppData\Local\Temp\tmp5z4e96le\assets
Epoch 1/10
2071/2071 [==============================] - 11s 5ms/step - loss: 1.4329 - categorical_crossentropy: 1.4329 - mean_squared_error: 0.1439 - accuracy: 0.4678
Epoch 2/10
2071/2071 [==============================] - 10s 5ms/step - loss: 1.2529 - categorical_crossentropy: 1.2529 - mean_squared_error: 0.1238 - accuracy: 0.5933
Epoch 3/10
2071/2071 [==============================] - 11s 5ms/step - loss: 1.1259 - categorical_crossentropy: 1.1259 - mean_squared_error: 0.1094 - accuracy: 0.6348
Epoch 4/10
2071/2071 [==============================] - 10s 5ms/step - loss: 1.0333 - categorical_crossentropy: 1.0333 - mean_squared_error: 0.0999 - accuracy: 0.6973
Epoch 5/10
2071/2071 [==============================] - 11s 5ms/step - loss: 0.9667 - categorical_crossentropy: 0.9668 - mean_squared_error: 0.0938 - accuracy: 0.7155
Epoch 6/10
2071/2071 [==============================] - 10s 5ms/step - loss: 0

In [29]:
for res in results:
    print(res['condition'])
    print_scores(res['score'])
    print('='*30)

{'lr': 0.001, 'm': 0.2}
Cross Entropy: 0.794 (+/- 0.001)
MSE: 0.105 (+/- 0.000)
Accuracy: 0.737 (+/- 0.001)
{'lr': 0.001, 'm': 0.6}
Cross Entropy: 0.654 (+/- 0.001)
MSE: 0.088 (+/- 0.001)
Accuracy: 0.781 (+/- 0.001)
{'lr': 0.05, 'm': 0.6}
Cross Entropy: 0.118 (+/- 0.002)
MSE: 0.014 (+/- 0.000)
Accuracy: 0.966 (+/- 0.001)
{'lr': 0.1, 'm': 0.6}
Cross Entropy: 0.095 (+/- 0.002)
MSE: 0.011 (+/- 0.000)
Accuracy: 0.972 (+/- 0.000)


In [31]:
history = results[0]["keras"]
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch')
plt.legend(['train_loss', 'val_loss', 'train_accuracy', 'val_accuracy'], loc='upper right')
plt.show()